
    ## 导学单元

    ### 课程目标
    - 回顾Budyko框架涉及的水热概念，理解综合分析如何将多源数据与模拟结合。
    - 熟悉Notebook与脚本 `examples/complex_integrated_analysis.py` 的配合方式，便于在命令行与交互环境间切换。

    ### 前置知识
    - 基础 Python（NumPy/Pandas）与 Matplotlib/Seaborn 绘图经验。
    - 对能量-水量平衡、蒸散发与降水等基本水文气候概念的认知。

    ### 所需包和运行环境
    - Python 3.9+，建议在虚拟环境或 Conda 环境中运行 Jupyter Notebook。
    - 依赖包主要列于项目 `requirements.txt`，常用库包括 NumPy、Pandas、Matplotlib、Seaborn、Xarray、SciPy（按需）。
    - 如需加速或批量运行，可直接在命令行调用 `examples/complex_integrated_analysis.py` 以生成对应输出。

    ### 学习路径与章节骨架
    1. 背景/概念：回顾Budyko框架涉及的水热概念，理解综合分析如何将多源数据与模拟结合。
    2. 数据加载与预览：理解数据来源、字段含义与基本统计特征。
    3. 核心方法步骤（配合 `examples/complex_integrated_analysis.py`）：依托脚本中的模块化函数执行数据模拟、指标计算、分区对比等集成操作。
    4. 结果可视化：用分面图、相关性矩阵与对比曲线呈现多情景结果，检查模型一致性。
    5. 反思与扩展练习：尝试替换模拟参数或引入额外气候情景，评估方法稳健性。

    ### 你将学到什么
    - 在单一Notebook中组织多步骤水文气候分析的最佳实践。
- 如何重用脚本函数构建可复现的集成实验。
- 利用多种可视化手段比较不同区域或情景的Budyko响应。

    ### 本节完成标志
    - [ ] 完成从数据生成/加载到可视化的全链路运行并得到多图输出。
- [ ] 能描述每个情景或分区在Budyko空间中的差异及可能原因。


# complex_integrated_analysis.py Notebook

# 🌊 Budyko水文分析完整教程

欢迎来到Budyko水文分析的世界！本教程将带你一步步了解如何分析流域的水量平衡。

## 📚 你将学到什么？

1. **什么是Budyko框架？** - 理解流域水量平衡的基本原理
2. **数据准备** - 了解需要哪些气象和水文数据
3. **计算蒸发** - 学习如何估算流域的实际蒸发量
4. **绘制Budyko图** - 可视化流域的水文特征
5. **时间变化分析** - 发现流域在不同时期的变化
6. **影响因素归因** - 找出是什么改变了流域的水文状况

## 🎯 学习目标

完成本教程后，你将能够：
- 理解气候、植被、人类活动如何影响流域水量平衡
- 使用Python分析真实的水文数据
- 制作专业的科研图表
- 解释分析结果的物理意义

---

**提示**：本教程使用模拟数据演示，但所有方法都可以应用于真实流域数据。

## 步骤 2️⃣：生成模拟的流域数据

为了教学目的，我们先生成一个模拟流域的60年数据（1961-2020）。

**这个流域的特点：**
- 📍 位置：中纬度地区（北纬40度）
- 🌡️ 气候：年均温约15°C，逐渐变暖
- 🌧️ 降水：年均约900mm，有波动
- 🌱 植被：逐渐变绿（LAI增加）
- 💧 人类活动：灌溉用水逐年增加

让我们看看这些数据长什么样！

In [3]:
# 生成模拟数据
df = generate_complex_data()

# 显示前几行数据，看看都有什么
print("\n📊 数据预览（前5年）：")
print(df[['year', 'P', 'Q_obs', 'T_avg', 'LAI', 'CO2']].head())

# 显示数据的统计摘要
print("\n📈 数据统计摘要：")
print(df[['P', 'Q_obs', 'T_avg', 'LAI']].describe())

NameError: name 'generate_complex_data' is not defined

### 📊 可视化：看看数据的变化趋势

让我们用图表来直观地看看这60年间各个变量的变化。

In [ ]:
# 创建一个包含4个子图的图表
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 子图1：降水和径流的变化
ax1 = axes[0, 0]
ax1.plot(df['year'], df['P'], 'b-', label='降水 P', linewidth=2)
ax1.plot(df['year'], df['Q_obs'], 'r-', label='观测径流 Q', linewidth=2)
ax1.set_xlabel('年份')
ax1.set_ylabel('水量 (mm/年)')
ax1.set_title('降水与径流的时间变化')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 子图2：温度变化
ax2 = axes[0, 1]
ax2.plot(df['year'], df['T_avg'], 'orange', linewidth=2)
ax2.set_xlabel('年份')
ax2.set_ylabel('温度 (°C)')
ax2.set_title('年均温度变化（注意变暖趋势）')
ax2.grid(True, alpha=0.3)

# 子图3：植被变化（LAI）
ax3 = axes[1, 0]
ax3.plot(df['year'], df['LAI'], 'green', linewidth=2)
ax3.set_xlabel('年份')
ax3.set_ylabel('叶面积指数 LAI')
ax3.set_title('植被绿化趋势（LAI增加）')
ax3.grid(True, alpha=0.3)

# 子图4：CO2浓度变化
ax4 = axes[1, 1]
ax4.plot(df['year'], df['CO2'], 'purple', linewidth=2)
ax4.set_xlabel('年份')
ax4.set_ylabel('CO₂浓度 (ppm)')
ax4.set_title('大气CO₂浓度上升')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 观察要点：")
print("  • 降水有波动但无明显趋势")
print("  • 径流逐渐减少（为什么？）")
print("  • 温度在上升（气候变暖）")
print("  • 植被在增加（绿化）")
print("  • CO₂浓度持续上升")

## 步骤 3️⃣：计算潜在蒸发量（PET）

**什么是潜在蒸发量？**
想象一个水盆放在太阳下，在水分充足的情况下，它能蒸发多少水？这就是潜在蒸发量（PET）。

**两种计算方法：**
1. **标准方法**：Penman-Monteith公式（FAO-56）
2. **创新方法**：考虑植被（LAI）和CO₂影响的改进公式

**为什么要考虑CO₂？**
- CO₂浓度升高 → 植物气孔部分关闭 → 蒸腾减少 → PET降低

In [ ]:
# 计算两种PET
df = step2_pet_comparison(df)

# 对比两种方法的结果
print(f"\n📊 PET对比结果：")
print(f"  标准方法（PM-FAO56）：{df['PET_standard'].mean():.1f} mm/年")
print(f"  创新方法（LAI+CO₂）： {df['PET_lai_co2'].mean():.1f} mm/年")
print(f"  差异：{(df['PET_lai_co2'] - df['PET_standard']).mean():.1f} mm/年")

# 绘制对比图
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 左图：时间序列对比
ax1.plot(df['year'], df['PET_standard'], 'gray', label='标准方法', linewidth=2)
ax1.plot(df['year'], df['PET_lai_co2'], 'green', label='创新方法（LAI+CO₂）', linewidth=2)
ax1.set_xlabel('年份')
ax1.set_ylabel('PET (mm/年)')
ax1.set_title('两种PET计算方法的时间序列对比')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 右图：散点图对比
ax2.scatter(df['PET_standard'], df['PET_lai_co2'], alpha=0.6, s=30)
ax2.plot([500, 1500], [500, 1500], 'r--', label='1:1线')
ax2.set_xlabel('标准方法 PET (mm/年)')
ax2.set_ylabel('创新方法 PET (mm/年)')
ax2.set_title('两种方法的相关性')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 思考：")
print("  如果创新方法的PET更低，可能是因为CO₂上升抑制了蒸发")
print("  这会影响我们对流域水量平衡的理解")

## 步骤 4️⃣：理解Budyko框架

**Budyko框架是什么？**

这是一个经典的水文理论，用来理解流域的水量平衡：

$$P = EA + Q + \Delta S$$

其中：
- **P**：降水（输入）
- **EA**：实际蒸发（输出1）
- **Q**：径流（输出2）
- **ΔS**：储量变化（地下水、土壤水等）

**两个关键指数：**

1. **干旱指数（IA）**：$IA = \frac{PET}{P}$ 
   - 衡量"能量充足程度"
   - IA > 1：能量充足，水分不足（干旱区）
   - IA < 1：水分充足，能量不足（湿润区）

2. **蒸发指数（IE）**：$IE = \frac{EA}{P}$
   - 衡量"降水中有多少被蒸发"
   - IE = 0.8 表示80%的降水被蒸发，20%形成径流

In [ ]:
# 计算Budyko指数
df, wb_results = step3_water_balance(df)

# 显示结果
print(f"\n📊 流域的平均Budyko指数：")
print(f"  干旱指数 IA = PET/P = {df['IA'].mean():.3f}")
print(f"  蒸发指数 IE = EA/P  = {df['IE'].mean():.3f}")
print(f"\n  解读：")
if df['IA'].mean() > 1:
    print(f"    • IA > 1：这是一个能量充足、水分受限的流域（偏干旱）")
else:
    print(f"    • IA < 1：这是一个水分充足、能量受限的流域（偏湿润）")
print(f"    • 平均{df['IE'].mean()*100:.1f}%的降水被蒸发掉了")
print(f"    • 平均{(1-df['IE'].mean())*100:.1f}%的降水形成了径流")

# 绘制经典的Budyko曲线
fig, ax = plt.subplots(figsize=(10, 8))

# 绘制理论曲线
ia_range = np.linspace(0.1, 5, 200)
for omega in [1.5, 2.0, 2.5, 3.0, 3.5]:
    ie_theory = BudykoCurves.tixeront_fu(ia_range, omega)
    ax.plot(ia_range, ie_theory, 'gray', linestyle='--', alpha=0.6, linewidth=1)
    # 标注omega值
    ax.text(ia_range[-1], ie_theory[-1], f'ω={omega}', fontsize=9, alpha=0.7)

# 绘制能量限制线和水分限制线
ax.plot([0, 5], [0, 1], 'b--', label='能量限制线 (IE=IA)', alpha=0.5)
ax.plot([0, 5], [1, 1], 'r--', label='水分限制线 (IE=1)', alpha=0.5)

# 绘制我们的流域数据点（用颜色表示年份）
scatter = ax.scatter(df['IA'], df['IE'], c=df['year'], cmap='viridis', 
                    s=50, alpha=0.7, edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, label='年份', ax=ax)

# 标注起点和终点
ax.scatter(df.iloc[0]['IA'], df.iloc[0]['IE'], c='blue', s=200, 
          marker='*', edgecolors='black', linewidth=2, label='起点(1961)', zorder=5)
ax.scatter(df.iloc[-1]['IA'], df.iloc[-1]['IE'], c='red', s=200, 
          marker='*', edgecolors='black', linewidth=2, label='终点(2020)', zorder=5)

ax.set_xlabel('干旱指数 IA = PET / P', fontsize=12, fontweight='bold')
ax.set_ylabel('蒸发指数 IE = EA / P', fontsize=12, fontweight='bold')
ax.set_title('Budyko空间：我们的流域在哪里？', fontsize=14, fontweight='bold')
ax.set_xlim(0, 5)
ax.set_ylim(0, 1.1)
ax.grid(True, alpha=0.3)
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()

print("\n💡 图表解读：")
print("  • 灰色虚线：理论Budyko曲线（不同的ω代表不同的流域特性）")
print("  • 彩色散点：我们流域60年的数据（颜色越深年份越早）")
print("  • 蓝色星：1961年的位置")
print("  • 红色星：2020年的位置")
print("  • 观察数据点是否沿着某条曲线分布，还是偏离了？")

## 步骤 5️⃣：分析时间变化 - Ibrahim方法

**研究问题**：流域的水文特性随时间稳定吗？

我们将60年分成3个时段，每个20年：
- **T1**：1961-1980（早期）
- **T2**：1981-2000（中期）
- **T3**：2001-2020（近期）

**核心思想**：
1. 用T1的数据拟合一条Budyko曲线（找到参数ω₁）
2. 看T2的数据是否还符合这条曲线（计算偏差ε）
3. 用T2的数据拟合新曲线（找到ω₂）
4. 看T3的数据是否符合T2的曲线

**如果有偏差，说明什么？**
- 可能是气候变化
- 可能是植被变化
- 可能是人类活动（如灌溉、水库）

In [ ]:
# 执行Ibrahim偏差分析
ibrahim_results = step4_ibrahim_analysis(df)

# 可视化三个时段
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

period_colors = {'T1': 'blue', 'T2': 'green', 'T3': 'red'}
period_names = {'T1': '早期(1961-1980)', 'T2': '中期(1981-2000)', 'T3': '近期(2001-2020)'}

ia_range = np.linspace(0.1, 5, 200)

for idx, (period_key, period_name) in enumerate([('T1', '早期(1961-1980)'), 
                                                   ('T2', '中期(1981-2000)'), 
                                                   ('T3', '近期(2001-2020)')]):
    ax = axes[idx]
    period_data = ibrahim_results['periods'][period_key]
    
    # 绘制该时段的数据点
    ax.scatter(period_data['ia_annual'], period_data['ie_annual'], 
              c=period_colors[period_key], s=50, alpha=0.7, 
              label=f'{period_name}数据点')
    
    # 绘制该时段拟合的曲线
    omega = period_data['omega']
    ie_fitted = BudykoCurves.tixeront_fu(ia_range, omega)
    ax.plot(ia_range, ie_fitted, color=period_colors[period_key], 
           linewidth=2, label=f'拟合曲线 (ω={omega:.2f})')
    
    # 标注均值点
    ax.scatter(period_data['ia_mean'], period_data['ie_mean'], 
              c=period_colors[period_key], s=300, marker='*', 
              edgecolors='black', linewidth=2, label='时段均值', zorder=5)
    
    ax.set_xlabel('干旱指数 IA')
    ax.set_ylabel('蒸发指数 IE')
    ax.set_title(f'{period_name}\nω = {omega:.3f}', fontweight='bold')
    ax.set_xlim(0, 5)
    ax.set_ylim(0, 1.1)
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=9)

plt.tight_layout()
plt.show()

# 显示偏差分析结果
print(f"\n📊 时段对比分析：")
print(f"  T1时段 ω = {ibrahim_results['periods']['T1']['omega']:.3f}")
print(f"  T2时段 ω = {ibrahim_results['periods']['T2']['omega']:.3f} (变化: {ibrahim_results['periods']['T2']['omega']-ibrahim_results['periods']['T1']['omega']:+.3f})")
print(f"  T3时段 ω = {ibrahim_results['periods']['T3']['omega']:.3f} (变化: {ibrahim_results['periods']['T3']['omega']-ibrahim_results['periods']['T2']['omega']:+.3f})")

print(f"\n📊 偏差分析：")
dev_T2 = ibrahim_results['deviations']['T1-T2']['distribution']
dev_T3 = ibrahim_results['deviations']['T2-T3']['distribution']
print(f"  T2相对T1的偏差中位数: {dev_T2.median:.4f}")
print(f"  T3相对T2的偏差中位数: {dev_T3.median:.4f}")
print(f"\n  时间稳定性评估: {ibrahim_results['stability']['category']}")

print("\n💡 解读：")
if abs(dev_T2.median) < 0.01 and abs(dev_T3.median) < 0.01:
    print("  • 偏差很小，流域特性稳定")
else:
    print("  • 偏差较大，流域特性发生了变化")
    print("  • 可能原因：气候变化、植被变化、人类活动")

## 步骤 6️⃣：轨迹分析 - Jaramillo方法

**研究问题**：流域从早期到近期是如何"移动"的？

我们比较T1（早期）和T3（近期）的平均位置：
- **起点**：T1的平均(IA, IE)
- **终点**：T3的平均(IA, IE)
- **运动向量**：从起点到终点的箭头

**判断标准**：
- 如果移动沿着Budyko曲线 → 自然变化（气候驱动）
- 如果移动偏离Budyko曲线 → 可能有人类活动影响

In [ ]:
# 执行Jaramillo轨迹分析
jaramillo_results = step5_jaramillo_analysis(ibrahim_results)

# 可视化运动轨迹
fig, ax = plt.subplots(figsize=(10, 8))

# 绘制理论曲线
ia_range = np.linspace(0.1, 5, 200)
for omega in [1.5, 2.0, 2.5, 3.0, 3.5]:
    ie_theory = BudykoCurves.tixeront_fu(ia_range, omega)
    ax.plot(ia_range, ie_theory, 'gray', linestyle='--', alpha=0.4, linewidth=1)

# 获取运动数据
mov = jaramillo_results['movement']

# 绘制所有数据点（浅色背景）
ax.scatter(df['IA'], df['IE'], c='lightgray', s=20, alpha=0.3, label='所有年份数据')

# 标注T1和T3的均值点
p1 = ibrahim_results['periods']['T1']
p3 = ibrahim_results['periods']['T3']

ax.scatter(p1['ia_mean'], p1['ie_mean'], c='blue', s=400, marker='o', 
          edgecolors='black', linewidth=3, label='T1 起点(1961-1980)', zorder=5)
ax.scatter(p3['ia_mean'], p3['ie_mean'], c='red', s=400, marker='s', 
          edgecolors='black', linewidth=3, label='T3 终点(2001-2020)', zorder=5)

# 绘制运动箭头
ax.annotate('', xy=(p3['ia_mean'], p3['ie_mean']), 
           xytext=(p1['ia_mean'], p1['ie_mean']),
           arrowprops=dict(arrowstyle='->', lw=3, color='purple', 
                          mutation_scale=30))

# 在箭头中点添加文字标注
mid_ia = (p1['ia_mean'] + p3['ia_mean']) / 2
mid_ie = (p1['ie_mean'] + p3['ie_mean']) / 2
ax.text(mid_ia, mid_ie + 0.05, f"运动强度: {mov.intensity:.3f}\n方向: {mov.direction_angle:.1f}°", 
       fontsize=11, ha='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
       fontweight='bold')

ax.set_xlabel('干旱指数 IA = PET / P', fontsize=12, fontweight='bold')
ax.set_ylabel('蒸发指数 IE = EA / P', fontsize=12, fontweight='bold')
ax.set_title('Jaramillo轨迹分析：流域如何演变？', fontsize=14, fontweight='bold')
ax.set_xlim(0, 5)
ax.set_ylim(0, 1.1)
ax.grid(True, alpha=0.3)
ax.legend(loc='lower right', fontsize=10)

plt.tight_layout()
plt.show()

# 显示结果
print(f"\n📊 轨迹分析结果：")
print(f"  起点 (T1): IA = {mov.ia_t1:.3f}, IE = {mov.ie_t1:.3f}")
print(f"  终点 (T3): IA = {mov.ia_t2:.3f}, IE = {mov.ie_t2:.3f}")
print(f"  ΔIA = {mov.delta_ia:+.3f}")
print(f"  ΔIE = {mov.delta_ie:+.3f}")
print(f"  运动强度 = {mov.intensity:.4f}")
print(f"  运动方向 = {mov.direction_angle:.1f}°")
print(f"  是否遵循曲线: {'是' if mov.follows_curve else '否'}")

print("\n💡 解读：")
if mov.follows_curve:
    print("  • 流域演变沿着Budyko曲线，符合自然规律")
    print("  • 主要是气候因素驱动")
else:
    print("  • 流域演变偏离了Budyko曲线！")
    print("  • 可能有显著的人类活动影响（如灌溉、水库）")
    
if mov.delta_ie > 0:
    print(f"  • IE增加 → 蒸发比例上升，径流比例下降")
else:
    print(f"  • IE减少 → 蒸发比例下降，径流比例上升")

## 步骤 7️⃣：找出原因 - 归因分析

**核心问题**：是什么导致了流域的偏差？

我们使用**机器学习**（随机森林）来找出最重要的驱动因子。

**方法**：
- **因变量Y**：Ibrahim方法计算的偏差（ε）
- **自变量X**：可能的驱动因子
  - 灌溉取水量
  - 植被LAI
  - CO₂浓度
  - 温度

**机器学习的优势**：
- 自动识别重要因子
- 量化每个因子的贡献
- 不需要事先假设因果关系

In [ ]:
# 执行归因分析
attribution_results = step6_attribution_analysis(df, ibrahim_results)

# 可视化归因结果
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# 左图：特征重要性条形图
importance = attribution_results.driver_importance
drivers = list(importance.keys())
values = list(importance.values())

# 按重要性排序
sorted_indices = np.argsort(values)[::-1]
drivers_sorted = [drivers[i] for i in sorted_indices]
values_sorted = [values[i] for i in sorted_indices]

# 中文标签映射
driver_labels = {
    'irrigation': '灌溉取水',
    'lai': '植被(LAI)',
    'co2': 'CO2浓度',
    'T_avg': '温度'
}
labels_sorted = [driver_labels.get(d, d) for d in drivers_sorted]

colors = ['red' if v > 0.3 else 'orange' if v > 0.15 else 'gray' for v in values_sorted]
ax1.barh(labels_sorted, values_sorted, color=colors, edgecolor='black', linewidth=1.5)
ax1.set_xlabel('特征重要性', fontsize=12, fontweight='bold')
ax1.set_title('各驱动因子的相对重要性', fontsize=13, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# 在条形图上标注数值
for i, v in enumerate(values_sorted):
    ax1.text(v + 0.01, i, f'{v:.1%}', va='center', fontweight='bold')

# 右图：解释性文字
ax2.axis('off')
explanation_text = f"""
归因分析结果解读

模型性能：
  R2 = {attribution_results.explained_variance:.1%}
  (表示模型能解释{attribution_results.explained_variance:.0%}的偏差变化)

驱动因子排序：
"""
for i, (label, value) in enumerate(zip(labels_sorted, values_sorted), 1):
    explanation_text += f"  {i}. {label}: {value:.1%}\n"

explanation_text += f"""
关键发现：
  最重要的因子是：{labels_sorted[0]}
  这意味着流域偏差主要由它驱动
  
  在我们的模拟中：
  - 灌溉取水逐年增加
  - 导致径流减少
  - 蒸发指数IE上升
  - 产生了偏离Budyko曲线的偏差
"""

ax2.text(0.1, 0.9, explanation_text, transform=ax2.transAxes, 
        fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"\n📊 归因分析详细结果：")
print(f"  模型性能 (R2): {attribution_results.explained_variance:.2%}")
print(f"\n  驱动因子重要性排序：")
for i, (driver, importance_val) in enumerate(sorted(importance.items(), 
                                                    key=lambda x: x[1], reverse=True), 1):
    print(f"    {i}. {driver_labels.get(driver, driver):12s}: {importance_val:.2%} {'⭐⭐⭐' if importance_val > 0.3 else '⭐⭐' if importance_val > 0.15 else '⭐'}")

print("\n💡 这告诉我们什么？")
print(f"  在近期(T3)，'{labels_sorted[0]}'是导致流域偏离理论曲线的主要原因")
print(f"  这为水资源管理提供了明确的方向")

## 步骤 8️⃣：生成综合分析报告

现在让我们把所有分析结果整合到一张图表中，制作一个专业的科研报告图！

In [ ]:
# 生成综合图表
step8_visualization(df, ibrahim_results, jaramillo_results, attribution_results)

print("\n🎉 恭喜！你已经完成了完整的Budyko水文分析！")
print("\n📚 你学到了：")
print("  ✓ 如何计算潜在蒸发量（PET）")
print("  ✓ 如何使用Budyko框架分析流域水量平衡")
print("  ✓ 如何评估流域特性的时间稳定性（Ibrahim方法）")
print("  ✓ 如何分析流域演变轨迹（Jaramillo方法）")
print("  ✓ 如何使用机器学习进行归因分析")
print("  ✓ 如何制作专业的科研图表")
print("\n🎯 下一步：")
print("  • 尝试将这些方法应用到真实流域数据")
print("  • 阅读相关文献，深入理解理论背景")
print("  • 探索更多的影响因子和分析方法")

---

## 📖 附录：关键概念总结

### 1. Budyko框架核心公式

水量平衡方程：
$$P = EA + Q + \Delta S$$

干旱指数：
$$IA = \frac{PET}{P}$$

蒸发指数：
$$IE = \frac{EA}{P} = \frac{P - Q}{P}$$

Fu方程（理论曲线）：
$$IE = 1 + IA - [1 + IA^\omega]^{1/\omega}$$

### 2. 物理意义

- **IA < 1**：湿润地区，水分充足，能量限制蒸发
- **IA > 1**：干旱地区，能量充足，水分限制蒸发
- **ω参数**：流域特性参数，受植被、土壤、地形影响

### 3. 分析方法对比

| 方法 | 研究问题 | 关键指标 |
|------|---------|---------|
| Ibrahim (2025) | 时间稳定性 | 偏差ε、稳定性类别 |
| Jaramillo (2022) | 演变轨迹 | 运动向量、方向角 |
| 随机森林 | 驱动归因 | 特征重要性 |

### 4. 常见流域类型

- **稳定型**：ω不变，沿曲线移动
- **渐变型**：ω缓慢变化，逐渐偏离
- **突变型**：ω突然变化，明显偏离

### 5. 实际应用

- 🌊 **水资源评估**：估算可用水量
- 🌡️ **气候变化影响**：评估未来水资源
- 🚜 **人类活动监测**：识别不可持续用水
- 🌳 **生态恢复**：评估植被恢复效果

## 🎓 延伸学习

### 推荐阅读

1. **经典文献**：
   - Budyko (1974) - 水热平衡理论的奠基之作
   - Fu (1981) - Fu方程的原始论文
   
2. **最新研究**：
   - Ibrahim et al. (2025) - 时间稳定性分析方法
   - Jaramillo et al. (2022) - 轨迹分析方法
   - He et al. (2023) - 3D Budyko框架

### 练习题

1. **基础题**：尝试修改模拟参数，观察结果如何变化
   - 增加灌溉强度会怎样？
   - 如果没有气候变暖趋势呢？

2. **进阶题**：将方法应用到真实流域
   - 下载一个流域的气象和水文数据
   - 重复本教程的分析流程
   - 解释你的发现

3. **挑战题**：改进现有方法
   - 加入更多驱动因子（如地下水开采）
   - 尝试其他机器学习模型
   - 考虑空间异质性

### 获取帮助

- 📧 项目Issues：遇到bug或有问题？提交Issue
- 💬 讨论区：想交流想法？加入Discussions
- 📚 文档：查看完整API文档

---

**祝你在水文研究的道路上一帆风顺！** 🌊🎓

## 步骤 1️⃣：导入必要的工具包

就像做实验需要准备仪器一样，我们首先要导入Python的各种工具包。

**这些工具包的作用：**
- `numpy`：处理数字和数组运算
- `pandas`：管理表格数据（类似Excel）
- `matplotlib`：绘制图表
- 我们自己开发的水文分析工具

In [ ]:
# examples/complex_integrated_analysis.py

import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 添加src路径 (Jupyter notebook 兼容)
from pathlib import Path
project_root = Path.cwd()
if project_root.name == 'examples':
    project_root = project_root.parent
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# 导入中文字体配置
try:
    from utils.plotting_config import setup_chinese_fonts
    setup_chinese_fonts()
    print("✓ 中文字体配置已应用")
except ImportError:
    # 备用配置
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
    print("✓ 使用备用中文字体配置")
import seaborn as sns

# 使用包名导入（而不是 src. 前缀）
from budyko.water_balance import WaterBalanceCalculator, WaterBalanceResults
from budyko.curves import BudykoCurves
from budyko.deviation import DeviationAnalysis, TemporalStability, MarginalDistribution
from budyko.trajectory_jaramillo import TrajectoryAnalyzer
from models.pet_lai_co2 import PETWithLAICO2
from models.pet_models import PenmanMonteith
from analysis.deviation_attribution import DeviationAttributor
from analysis.snow_analyzer import SnowImpactAnalyzer
from visualization.budyko_plots import BudykoVisualizer
from visualization.direction_rose import plot_direction_rose

print(f"✓ 环境准备完成")
print(f"项目根目录: {project_root}")
print(f"源代码路径: {src_path}")

### 📦 定义辅助函数

下面定义一些辅助函数，用于后续的数据生成和分析步骤。

**主要函数包括：**
- `generate_complex_data()` - 生成模拟流域数据
- `step2_pet_comparison()` - PET计算对比
- `step3_water_balance()` - 水量平衡计算
- `step4_ibrahim_analysis()` - Ibrahim偏差分析
- `step5_jaramillo_analysis()` - Jaramillo轨迹分析
- `step6_attribution_analysis()` - 归因分析
- `step8_visualization()` - 综合可视化

这些函数封装了复杂的计算逻辑，让主流程更清晰易懂。

In [ ]:
# --- 辅助函数定义 ---

# 1. 数据生成函数
def generate_complex_data(n_years: int = 60, start_year: int = 1961) -> pd.DataFrame:
    """生成一个复杂的、有趋势的模拟流域数据集"""
    print(f"[1. 数据模拟] 正在生成 {n_years} 年的模拟数据...")
    
    np.random.seed(42)
    years = np.arange(start_year, start_year + n_years)
    time_idx = np.arange(n_years)
    
    # 1. 气候变量
    P = 900 + 50 * np.sin(2 * np.pi * time_idx / 30) + 150 * np.random.randn(n_years)
    P = np.maximum(P, 400)
    T_avg = 15 + 0.03 * time_idx + 0.5 * np.random.randn(n_years)
    T_max = T_avg + 6
    T_min = T_avg - 6
    RH = 60 - 0.1 * time_idx + 5 * np.random.randn(n_years)
    u2 = 2.0 + 0.3 * np.random.randn(n_years)
    Rn_MJ = 7.0 + 0.5 * np.sin(2 * np.pi * time_idx / 20) + 0.5 * np.random.randn(n_years)
    Rn_W = Rn_MJ / 0.0864
    
    # 2. 创新驱动因子
    LAI = 2.5 + 0.02 * time_idx + 0.3 * np.random.randn(n_years)
    LAI = np.clip(LAI, 1.0, 6.0)
    CO2 = 320 + 1.5 * time_idx + 2 * np.random.randn(n_years)
    
    # 3. 储量变化
    TWS = 50 * np.sin(2 * np.pi * time_idx / 15) - 0.5 * time_idx
    delta_S = np.diff(TWS, prepend=TWS[0])
    
    # 4. 模拟人类活动（灌溉）
    irrigation_withdrawal = 10 + 1.2 * time_idx + 5 * np.random.randn(n_years)
    irrigation_withdrawal = np.maximum(irrigation_withdrawal, 0)

    # 5. 计算径流
    pet_calc = PETWithLAICO2(elevation=500, latitude=40)
    PET_lai_co2 = pet_calc.calculate(T_avg, RH, u2, Rn_W, LAI, CO2)
    PET_lai_co2_annual = PET_lai_co2 * 365.25
    
    IA_nat = PET_lai_co2_annual / P
    omega_nat = 2.5
    IE_nat = BudykoCurves.tixeront_fu(IA_nat, omega_nat)
    EA_nat = IE_nat * P
    Q_nat = P - EA_nat
    
    Q_obs = P - EA_nat - delta_S - irrigation_withdrawal
    Q_obs = np.maximum(Q_obs, Q_nat * 0.1)
    
    data = {
        'year': years, 'P': P, 'Q_obs': Q_obs, 'Q_nat': Q_nat,
        'T_avg': T_avg, 'T_max': T_max, 'T_min': T_min,
        'RH': RH, 'u2': u2, 'Rn_MJ': Rn_MJ, 'Rn_W': Rn_W,
        'LAI': LAI, 'CO2': CO2, 'TWS': TWS, 'delta_S': delta_S,
        'irrigation_withdrawal': irrigation_withdrawal,
        'PET_lai_co2': PET_lai_co2_annual
    }
    df = pd.DataFrame(data)
    print(f"  ✓ 模拟数据生成完毕")
    return df

# 2. PET对比函数
def step2_pet_comparison(df: pd.DataFrame) -> pd.DataFrame:
    """计算并对比两种PET方法"""
    print("\n[2. PET计算对比]")
    
    pet_standard_calc = PenmanMonteith()
    pet_standard = pet_standard_calc.calculate(
        temp_avg=df['T_avg'].to_numpy(),
        temp_max=df['T_max'].to_numpy(),
        temp_min=df['T_min'].to_numpy(),
        rh_mean=df['RH'].to_numpy(),
        wind_speed=df['u2'].to_numpy(),
        solar_radiation=df['Rn_MJ'].to_numpy(),
        latitude=40.0, elevation=500.0,
        day_of_year=np.tile(np.arange(1, 366), len(df)//365 + 1)[:len(df)]
    )
    pet_standard_annual = pd.Series(pet_standard).rolling(window=365, min_periods=365).sum().to_numpy()
    mean_ps = float(np.nanmean(pet_standard_annual))
    if not np.isfinite(mean_ps):
        mean_ps = float(np.nanmean(pet_standard)) if np.isfinite(np.nanmean(pet_standard)) else 0.0
    pet_standard_annual = np.where(np.isnan(pet_standard_annual), mean_ps, pet_standard_annual)
    
    pet_calc_lai_co2 = PETWithLAICO2(elevation=500, latitude=40)
    pet_lai_co2_daily = pet_calc_lai_co2.calculate(
        temperature=df['T_avg'].values, humidity=df['RH'].values,
        wind_speed=df['u2'].values, radiation=df['Rn_W'].values,
        lai=df['LAI'].values, co2=df['CO2'].values
    )
    pet_lai_co2_annual = pd.Series(pet_lai_co2_daily).rolling(window=365, min_periods=365).sum().to_numpy()
    mean_plc = float(np.nanmean(pet_lai_co2_annual))
    if not np.isfinite(mean_plc):
        mean_plc = float(np.nanmean(pet_lai_co2_daily)) if np.isfinite(np.nanmean(pet_lai_co2_daily)) else 0.0
    pet_lai_co2_annual = np.where(np.isnan(pet_lai_co2_annual), mean_plc, pet_lai_co2_annual)

    df['PET_standard'] = pet_standard_annual
    df['PET_lai_co2'] = pet_lai_co2_annual
    
    print(f"  ✓ PET计算完成")
    return df

# 3. 水量平衡函数
def step3_water_balance(df: pd.DataFrame):
    """计算2D和3D Budyko指数"""
    print("\n[3. 水量平衡计算]")
    
    wb_calc = WaterBalanceCalculator()
    wb_results = wb_calc.calculate_budyko_indices(
        P=df['P'].to_numpy(),
        Q=df['Q_obs'].to_numpy(),
        PET=df['PET_lai_co2'].to_numpy(),
        delta_S=df['delta_S'].to_numpy(),
        TWS=df['TWS'].to_numpy()
    )
    
    df['IA'] = wb_results.aridity_index
    df['IE'] = wb_results.evaporation_index
    df['EA'] = wb_results.actual_evaporation
    df['IE_ext'] = wb_results.evaporation_index_extended
    df['SCI'] = wb_results.storage_change_index
    
    print(f"  ✓ 水量平衡计算完成")
    return df, wb_results

# 4. Ibrahim偏差分析函数
def step4_ibrahim_analysis(df: pd.DataFrame):
    """执行Ibrahim偏差分析"""
    print("\n[4. Ibrahim 偏差分析]")
    
    deviation_analyzer = DeviationAnalysis(period_length=20)
    stability_analyzer = TemporalStability()
    marginal_analyzer = MarginalDistribution()
    
    periods_data = {}
    deviation_dists = {}
    period_defs = [('T1', 1961, 1980), ('T2', 1981, 2000), ('T3', 2001, 2020)]
    
    for name, start, end in period_defs:
        period_df = df[(df['year'] >= start) & (df['year'] <= end)]
        omega, stats = BudykoCurves.fit_omega(period_df['IA'].to_numpy(), period_df['IE'].to_numpy())
        periods_data[name] = {
            'omega': omega,
            'ia_annual': period_df['IA'].to_numpy(),
            'ie_annual': period_df['IE'].to_numpy(),
            'ia_mean': period_df['IA'].mean(),
            'ie_mean': period_df['IE'].mean(),
            'start_year': start,
            'end_year': end
        }
        
    dist_T2 = deviation_analyzer.calculate_deviations(
        ia_i=periods_data['T1']['ia_annual'],
        ie_obs_i=periods_data['T1']['ie_annual'],
        omega_i=periods_data['T1']['omega'],
        ia_i_plus_1=periods_data['T2']['ia_annual'],
        ie_obs_i_plus_1=periods_data['T2']['ie_annual'],
        period_pair='T1-T2'
    )
    wilcoxon_T2 = deviation_analyzer.wilcoxon_test(dist_T2)
    deviation_dists['T1-T2'] = {'distribution': dist_T2, 'wilcoxon_test': wilcoxon_T2}

    dist_T3 = deviation_analyzer.calculate_deviations(
        ia_i=periods_data['T2']['ia_annual'],
        ie_obs_i=periods_data['T2']['ie_annual'],
        omega_i=periods_data['T2']['omega'],
        ia_i_plus_1=periods_data['T3']['ia_annual'],
        ie_obs_i_plus_1=periods_data['T3']['ie_annual'],
        period_pair='T2-T3'
    )
    wilcoxon_T3 = deviation_analyzer.wilcoxon_test(dist_T3)
    deviation_dists['T2-T3'] = {'distribution': dist_T3, 'wilcoxon_test': wilcoxon_T3}
    
    stability = stability_analyzer.assess_stability(
        distributions=[dist_T2, dist_T3],
        ie_means=[periods_data['T2']['ie_mean'], periods_data['T3']['ie_mean']]
    )
    
    marginal = marginal_analyzer.aggregate_distributions(
        [dist_T2, dist_T3], stability['category']
    )
    
    print(f"  ✓ Ibrahim分析完成")
    return {'periods': periods_data, 'stability': stability, 'marginal': marginal, 'deviations': deviation_dists}

# 5. Jaramillo轨迹分析函数
def step5_jaramillo_analysis(ibrahim_results):
    """执行Jaramillo轨迹分析"""
    print("\n[5. Jaramillo 轨迹分析]")
    
    trajectory_analyzer = TrajectoryAnalyzer()
    p1 = ibrahim_results['periods']['T1']
    p3 = ibrahim_results['periods']['T3']

    movement = trajectory_analyzer.calculate_movement(
        catchment_id='complex_basin',
        period_1={'IA': p1['ia_mean'], 'IE': p1['ie_mean'], 'name': 'T1'},
        period_2={'IA': p3['ia_mean'], 'IE': p3['ie_mean'], 'name': 'T3'}
    )
    
    print(f"  ✓ Jaramillo分析完成")
    return {'movement': movement}

# 6. 归因分析函数
def step6_attribution_analysis(df: pd.DataFrame, ibrahim_results):
    """执行偏差归因分析"""
    print("\n[6. 偏差归因分析]")
    
    attributor = DeviationAttributor()
    y_deviation = ibrahim_results['deviations']['T2-T3']['distribution'].annual_deviations
    df_T3 = df[df['year'].isin(range(2001, 2021))]
    
    attributor.set_deviation(y_deviation)
    attributor.add_driver('irrigation', df_T3['irrigation_withdrawal'].to_numpy())
    attributor.add_driver('lai', df_T3['LAI'].to_numpy())
    attributor.add_driver('co2', df_T3['CO2'].to_numpy())
    attributor.add_driver('T_avg', df_T3['T_avg'].to_numpy())
    
    rf_results = attributor.random_forest_attribution()
    
    print(f"  归因模型 R2: {rf_results.explained_variance:.2%}")
    for driver, importance in sorted(rf_results.driver_importance.items(), key=lambda x: x[1], reverse=True):
        print(f"    - {driver:15s}: {importance:.2%}")
    
    return rf_results

# 8. 综合可视化函数
def step8_visualization(df: pd.DataFrame, ibrahim, jaramillo, attribution):
    """生成综合图表"""
    print("\n[8. 综合可视化]")
    
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(3, 3)
    
    # 图A: Budyko空间
    ax_a = fig.add_subplot(gs[0, 0])
    sns.scatterplot(data=df, x='IA', y='IE', hue='year', palette='viridis', s=20, ax=ax_a, legend=False)
    ia_range = np.linspace(0, 5, 200)
    for om in [2.0, 2.5, 3.0]:
        ax_a.plot(ia_range, BudykoCurves.tixeront_fu(ia_range, om), linestyle='--', alpha=0.6)
    ax_a.set_title("A: Budyko空间", fontweight='bold')
    ax_a.set_xlabel("干旱指数 IA")
    ax_a.set_ylabel("蒸发指数 IE")
    
    # 图B: Ibrahim偏差分布
    ax_b = fig.add_subplot(gs[0, 1])
    dev_T2 = ibrahim['deviations']['T1-T2']['distribution'].annual_deviations
    dev_T3 = ibrahim['deviations']['T2-T3']['distribution'].annual_deviations
    sns.histplot(dev_T2, kde=True, ax=ax_b, color='blue', alpha=0.5, label='T2')
    sns.histplot(dev_T3, kde=True, ax=ax_b, color='red', alpha=0.5, label='T3')
    ax_b.set_title(f"B: Ibrahim偏差 ({ibrahim['stability']['category']})", fontweight='bold')
    ax_b.set_xlabel("偏差")
    ax_b.legend()
    
    # 图C: Jaramillo轨迹
    ax_c = fig.add_subplot(gs[0, 2])
    mov = jaramillo['movement']
    ax_c.arrow(mov.ia_t1, mov.ie_t1, mov.delta_ia, mov.delta_ie, 
             head_width=0.01, fc='r', ec='r')
    ax_c.plot(mov.ia_t1, mov.ie_t1, 'bo', label='T1')
    ax_c.plot(mov.ia_t2, mov.ie_t2, 'rs', label='T3')
    ax_c.set_title("C: Jaramillo轨迹", fontweight='bold')
    ax_c.set_xlabel("IA")
    ax_c.set_ylabel("IE")
    ax_c.legend()
    
    # 图D-I: 其他分析
    ax_d = fig.add_subplot(gs[1, 0])
    sns.scatterplot(data=df, x='IA', y='IE_ext', hue='SCI', palette='coolwarm', s=20, ax=ax_d, legend=False)
    ax_d.set_title("D: 3D Budyko", fontweight='bold')
    
    ax_e = fig.add_subplot(gs[1, 1])
    sns.regplot(data=df, x='IA', y='SCI', ax=ax_e, scatter_kws={'s': 10})
    ax_e.set_title("E: DI vs SCI", fontweight='bold')
    
    ax_f = fig.add_subplot(gs[1, 2])
    imp = attribution.driver_importance
    drivers = sorted(imp.keys(), key=lambda k: imp[k], reverse=True)
    importances = [imp[k] for k in drivers]
    sns.barplot(x=importances, y=drivers, ax=ax_f, orient='h')
    ax_f.set_title("F: 偏差归因", fontweight='bold')
    
    ax_g = fig.add_subplot(gs[2, 0])
    ax_g.plot(df['year'], df['PET_standard'], label='标准PET')
    ax_g.plot(df['year'], df['PET_lai_co2'], label='创新PET')
    ax_g.set_title("G: PET对比", fontweight='bold')
    ax_g.legend()
    
    ax_h = fig.add_subplot(gs[2, 1])
    ax_h2 = ax_h.twinx()
    ax_h.plot(df['year'], df['LAI'], 'g-', label='LAI')
    ax_h2.plot(df['year'], df['CO2'], 'k-', label='CO2')
    ax_h.set_title("H: 驱动因子趋势", fontweight='bold')
    
    ax_i = fig.add_subplot(gs[2, 2])
    ax_i.plot(df['year'], df['P'], label='P')
    ax_i.plot(df['year'], df['Q_obs'], label='Q')
    ax_i.plot(df['year'], df['EA'], label='EA')
    ax_i.set_title("I: 水量平衡", fontweight='bold')
    ax_i.legend()

    plt.tight_layout(pad=1.5)
    figures_dir = Path.cwd().parent / "examples" / "figures" if Path.cwd().name != "examples" else Path.cwd() / "figures"
    figures_dir.mkdir(parents=True, exist_ok=True)
    save_path = figures_dir / "complex_integrated_analysis__summary.png"
    plt.savefig(save_path, dpi=300)
    print(f"  ✓ 综合图表已保存: {save_path}")
    plt.close()

print("✓ 所有辅助函数定义完成")

---

现在所有准备工作完成了，让我们开始实际的分析流程！

请按顺序运行下面的单元格，每个步骤都会有详细的解释和可视化。